This notebook plots the depth and area of various imaging surveys. Where possible, limits are $r$ band, or a wider band that covers the $r$ band. 
Some limits are point sources, and some are extended sources. 
That means the numbers here aren't super precise, but the overall hierarchies and gist of the plot are the same.

This plot is based off the similar plot on the [HSC website weak lensing page](https://hsc.mtk.nao.ac.jp/ssp/science/weak-lensing-cosmology/).
Some of my numbers are taken directly from that plot.

LSST sources:
- DDFs: https://project.lsst.org/groups/sac/sites/lsst.org.groups.sac/files/Brandt_DDF.pdf
- Commissioning: https://project.lsst.org/meetings/rubin2022/sites/default/files/Rubin%20PCW%202022%20-%20Science%20Verification%20and%20Validation_0.pdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
future_color = "C3"
ongoing_color = "C0"
complete_color = "k"

surveys = {
    # name: [area, depth, color]

    # Future
    "LSST\n(deep comm)": [100, 27.7, future_color],
    "LSST\n(wide comm)": [1_000, 26.4, future_color],
    "LSST\n(deep)": [40, 28.9, future_color],
    "LSST\n(wide)": [18_000, 27.3, future_color],
    #"Euclid\n(wide)": [15_000, 25.3, future_color],
    #"Euclid\n(deep)": [40, 27.3, future_color],

    # Ongoing
    "HSC\n(wide)": [1400, 26.1, ongoing_color],
    "HSC\n(deep)": [27, 27.1, ongoing_color],
    "HSC\n(UD)": [3.5, 27.7, ongoing_color],
    "DES\n(wide)": [5000, 25.2, ongoing_color],
    "DES\n(deep)": [5.88, 27.1, ongoing_color],
    "KiDS": [1500, 25.2, ongoing_color],
    "PS1\n($3\pi$)": [30_000, 23.2, ongoing_color],
    "PS1\n(MDS)": [70, 26, ongoing_color],
    "PS2": [7000, 24.9, ongoing_color],
    
    # Completed
    "SDSS\n(Legacy)": [7500, 23, complete_color],
    "SDSS\n(Stripe 82)": [300, 25.1, complete_color],
    "CFHTLS\n(deep)": [4, 26.6, complete_color],
    "CFHTLS\n(wide)": [155, 24.8, complete_color],
    "COSMOS": [2, 26.7, complete_color],
    "BCS": [80, 24.8, complete_color],
    "RCS": [100, 24.7, complete_color],
    "RCS-2": [1000, 24.3, complete_color],
    "NDWFS": [18, 26.0, complete_color],
    "SXDS": [1.3, 27.5, complete_color],
    #"DLS": [20, 27, complete_color],
    #"HDF": [0.00294, 29, complete_color],
    #"HUDF": [0.003194, 30, complete_color],
}

In [ ]:
def plot_surveys(lsst_full=False, euclid=False):
    fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
    ax.set(
        xlabel="Area (deg$^2$)",
        ylabel="Depth (AB mag, $5\sigma$)",
        xscale="log",
        xlim=(0.7, 5e4),
        ylim=(22.8, 29.7),
    )

    # plot the surveys
    for survey, (area, depth, color) in surveys.items():
        # skip the LSST full surveys?
        if not lsst_full and (survey == "LSST\n(deep)" or survey == "LSST\n(wide)"):
            continue
        # skip Euclid?
        if not euclid and "Euclid" in survey:
            continue
        
        # plot the survey point
        ax.scatter(area, depth, marker="s", s=10, c=color)

        # determine annotation offsets
        if survey == "Euclid\n(deep)":
            x_offset = 50
        elif survey == "CFHTLS\n(wide)" and "RCS" in surveys:
            x_offset = 90
        elif survey == "RCS" and "CFHTLS\n(wide)" in surveys:
            x_offset = -20
        else:
            x_offset = 0
        
        if survey in ["CFHTLS\n(wide)", "CFHTLS\n(deep)", "HSC\n(wide)", "HSC\n(deep)"]:
            y_offset = -0.55
        elif survey in ["PS2", "NDWFS", "KiDS"]:
            y_offset = -0.32
        elif survey == "RCS":
            y_offset = -0.31
        elif survey == "Euclid\n(deep)":
            y_offset = -0.25
        else:
            y_offset = 0.1

        # set box around text
        if "LSST" in survey and not ("comm" in survey and lsst_full):
            bbox = dict(facecolor="none", edgecolor="C3", pad=1, lw=0.5)
            y_offset += 0.07
        else:
            bbox = None

        # label the survey point
        ax.annotate(survey, (area+x_offset, depth+y_offset), fontsize=7, ha="center", va="bottom", color=color, bbox=bbox)

    # if we plotted the full LSST surveys, point from commissioning to science fields
    if lsst_full:
        pass
        ax.annotate(
            "",
            xy=np.array(surveys["LSST\n(deep)"][:2]) + np.array([+3, -0.05]),
            xytext=np.array(surveys["LSST\n(deep comm)"][:2]) + np.array([-30, +0.6]),
            arrowprops=dict(arrowstyle="->", color="C3"),
        )
        ax.annotate(
            "",
            xy=np.array(surveys["LSST\n(wide)"][:2]) + np.array([-1_500, -0.02]),
            xytext=np.array(surveys["LSST\n(wide comm)"][:2]) + np.array([1_000, 0.45]),
            arrowprops=dict(arrowstyle="->", color="C3"),
        )

    # put the legend in the bottom left
    ax.text(0.01, 0.01, "Completed", color="k", transform=ax.transAxes, va="bottom", ha="left", fontsize=9)
    ax.text(0.01, 0.06, "Ongoing", color="C0", transform=ax.transAxes, va="bottom", ha="left", fontsize=9)
    ax.text(0.01, 0.11, "Future", color="C3", transform=ax.transAxes, va="bottom", ha="left", fontsize=9)

    plt.tight_layout()

    return fig, ax

In [ ]:
fig, ax = plot_surveys(lsst_full=False, euclid=True)
fig.savefig("figures/surveys_preLsst.png", dpi=500)

In [ ]:
fig, ax = plot_surveys(lsst_full=True, euclid=True)
fig.savefig("figures/surveys_postLsst.png", dpi=500)